# Building a Basic Brick

In this tutorial, we'll go through a simple example of how to build a basic AND brick.  We'll start with some basic imports.  We almost always use NetworkX and Numpy.  The fugu imports are really just for testing our new brick.

In [6]:
import networkx as nx
import numpy as np
import fugu
from fugu import Scaffold, Brick
from fugu.bricks import Vector_Input

All bricks should inherit from the `Brick` class.  Bricks that are listed as input bricks should instead inherit from `InputBrick`, which is beyond the scope of this tutorial.  The construction of most brick types is similar; constructing a brick that takes input coding "current" (see below) is a bit different and is beyond the scope of this tutorial.

The `Brick` class provides the framework for the a scaffold to build a neural graph. Subclasses of `Brick` should provide the actual code that will generate the nodes and edges on a graph.  The graph construction should take place within the `build` method.  Let's look at the definition of the parent class `Brick`.

```python 
class Brick(ABC):
    def __init__(self):
        self.name = "Empty Brick"
        self.supported_codings = []

    @abstractmethod
    def build(self, graph,
                   metadata,
                   complete_node,
                   input_lists,
                   input_codings):
        pass

```

The first line `class Brick(ABC)` defines the abstract class of `Brick`.  Brick objects inherit from `ABC` which just means that `Brick` is an abstract class that cannot be instantiated on its own; only subclasses may be instantiated.

The ``__init__`` method contains standard instantiation code.  All bricks are expected to have a member property `self.name` that is unique to the brick.  The uniqueness needs to be determined by the scaffold, not by the brick.

The property `self.is_built` is a boolean that is True if the brick has been built (added to the graph).

The property `self.supported_codings` is a list of input codings (strings) that the brick supports.  Since you have the full use of python when you are defining your brick, you can support multiple coding types completely transparent to the user.  A full list of coding types can be found at `fugu.input_coding_types`.

The method `build` will be called by the scaffold when the graph is to be built.  Arguments are:
- graph: The graph object that we are building onto.
- metadata: A dictionary of shapes and parameters. This will likely be removed in future implementations, so don't rely on it.
- control_nodes: A *dict* of *lists* of nodes that transmit a control information.  The most common is `control_nodes['complete']` which carries a list of 'finished' spikes from input bricks.  If your brick has one input, then this will be a list of a single node.  The only other currently used key is `control_nodes['begin']` which is used for temporally coded bricks (and outside the scope of this tutorial)
- input_lists:  A *list of lists* of nodes that correspond to input neurons. The outermost list contains a list of neurons, one for each input on the scaffold.  
- input_codings: A *list* of input coding types. The list contains one coding type per input on the scaffold.

Each brick is responsible for throwing the appropriate errors/warnings if the inputs are not compatible with the brick.

Below is a basic AND brick.

In [7]:
class basic_AND(Brick):
    def __init__(self, name=None):
        super(Brick, self).__init__()
        #The brick hasn't been built yet.
        self.is_built = False
        #Leave for compatibility, D represents the depth of the circuit.  Needs to be updated.
        self.metadata = {'D':1}  
        #We just store the name passed at construction.
        self.name = name
        #For this example, we'll let any input coding work even though the answer might not make sense.
        self.supported_codings = fugu.input_coding_types
    def build(self,
             graph,
             metadata,
             control_nodes,
             input_lists,
             input_codings):
        #Expect two inputs
        if len(input_codings)!=2:
            raise ValueError('Only two inputs supported.')
        #Keep the same coding as input 0 for the output
        #This is an arbitrary decision at this point.  
        #Generally, your brick will impart some coding, but that isn't the case here.
        output_codings = [input_codings[0]]
        
        #All bricks should provide a neuron that spikes when the brick has completed processing.
        #We just put in a basic relay neuron that will spike when it receives any spike from its
        #single input, which is the complete_node from the first input.
        #All nodes we add to the graph should have basic neuron parameters (threshold, decay)
        #Reasonable defaults will be filled-in, but these defaults may depend on the execution platform.
        #Additionally, nodes should have a field called 'index' which is a local index used to reference the 
        #position of the node.  This can be used by downstream bricks.  A simple example might be
        #a 3-bit binary representation will add 3 nodes to the graph with indices 0,1,2
        #We do have to do some work to establish best practices here.
        new_complete_node_name = self.name + '_complete'
        graph.add_node(new_complete_node_name,
                      index = -1,
                      threshold = 0.0,
                      decay =0.0,
                      p=1.0,
                      potential=0.0)
        graph.add_edge(control_nodes[0]['complete'], new_complete_node_name,weight=1.0,delay=1)
        
        #We also, obviously, need to build the computational portion of our graph
        and_node_name = self.name + '_0'
        graph.add_node(and_node_name,
                       index=0,
                       threshold=1.0,
                       decay=1.0,
                       p=1.0,
                       potential=0.0
                      )
        graph.add_edge(input_lists[0][0],
                      and_node_name,
                      weight=0.75,
                      delay=1.0)
        graph.add_edge(input_lists[1][0],
                      and_node_name,
                      weight=0.75,
                      delay=1.0)
        self.is_built=True
        
        #Remember, bricks can have more than one output, so we need a list of list of output neurons
        output_lists = [[and_node_name]]
        
        return (graph,
               self.metadata,
                [{'complete':new_complete_node_name}],
                output_lists,
                output_codings
               )

We can then build a basic example scaffold and test it out.

In [5]:
scaffold = Scaffold()
scaffold.add_brick(Vector_Input(np.array([1]), coding='Raster', name='Input0'), 'input' )
scaffold.add_brick(Vector_Input(np.array([1]), coding='Raster', name='Input1'), 'input' )
scaffold.add_brick(basic_AND(name='AND'), [(0,0), (1,0)], output=True)
scaffold.lay_bricks()
result = scaffold.evaluate(backend='ds')
print(result)

AttributeError: 'basic_AND' object has no attribute 'brick_tag'

We can also use the indices stored on the nodes to make more sophisticated graphs:

In [4]:
class better_AND(Brick):
    def __init__(self, name=None):
        super(Brick, self).__init__()
        #The brick hasn't been built yet.
        self.is_built = False
        #Leave for compatibility, D represents the depth of the circuit.  Needs to be updated.
        self.metadata = {'D':1}  
        #We just store the name passed at construction.
        self.name = name
        #For this example, we'll let any input coding work even though the answer might not make sense.
        self.supported_codings = fugu.input_coding_types
    def build(self,
             graph,
             metadata,
             control_nodes,
             input_lists,
             input_codings):
        #Expect two inputs
        if len(input_codings)!=2:
            raise ValueError('Only two inputs supported.')
        #Keep the same coding as input 0 for the output
        #This is an arbitrary decision at this point.  
        #Generally, your brick will impart some coding, but that isn't the case here.
        output_codings = [input_codings[0]]
        
        #All bricks should provide a neuron that spikes when the brick has completed processing.
        #We just put in a basic relay neuron that will spike when it recieves any spike from its
        #single input, which is the complete_node from the first input.
        #All nodes we add to the graph should have basic neuron parameters (threshold, decay)
        #Reasonable defaults will be filled-in, but these defaults may depend on the execution platform.
        #Additionally, nodes should have a field called 'index' which is a local index used to reference the 
        #position of the node.  This can be used by downstream bricks.  A simple example might be
        #a 3-bit binary representation will add 3 nodes to the graph with indices 0,1,2
        #We do have to do some work to establish best practices here.
        new_complete_node_name = self.name + '_complete'
        graph.add_node(new_complete_node_name,
                      index = -1,
                      threshold = 0.0,
                      decay =0.0,
                      p=1.0,
                      potential=0.0)
        graph.add_edge(control_nodes[0]['complete'], new_complete_node_name,weight=1.0,delay=1)
        
        ##########################################################################
        #           Changes start here                                           #
        ##########################################################################
        output_lists = [[]]
        #We also, obviously, need to build the computational portion of our graph
        for operand0 in input_lists[0]:
            for idx_num, operand1 in enumerate(input_lists[1]):
                #If indices match, we'll do an AND on them
                if graph.nodes[operand0]['index'] == graph.nodes[operand1]['index']:
                    #Remember all of our output neurons need to be marked
                    and_node_name = self.name+ '_' + str(operand0) + '_' +str(operand1)
                    output_lists[0].append(and_node_name)
                    graph.add_node(and_node_name,
                       index=0,
                       threshold=1.0,
                       decay=1.0,
                       p=1.0,
                       potential=0.0
                      )
                    graph.add_edge(operand0,
                                  and_node_name,
                                  weight=0.75,
                                  delay=1.0)
                    graph.add_edge(operand1,
                                  and_node_name,
                                  weight=0.75,
                                  delay=1.0)
        self.is_built=True
        ########################################################################
        #             Changes end here                                         #
        ########################################################################
        
        
        return (graph,
               self.metadata,
                [{'complete':new_complete_node_name}],
                output_lists,
                output_codings
               )

And we can provide a better example:

In [6]:
scaffold = Scaffold()
scaffold.add_brick(Vector_Input(np.array([1,0,1,0]), coding='Raster', name='Input0'), 'input' )
scaffold.add_brick(Vector_Input(np.array([1,1,0,0]), coding='Raster', name='Input1'), 'input' )
scaffold.add_brick(better_AND(name='AND'), [(0,0), (1,0)], output=True)
scaffold.lay_bricks()
result = scaffold.evaluate(backend='ds')
print(result)

AttributeError: 'better_AND' object has no attribute 'brick_tag'

Right now it is a little difficult to debug, because we need to decode the neuron numbers by hand.  In the future, we should be able to automate this with just a little work. Until then, you can get the list of neurons with the following one line.

In [ ]:
[scaffold.graph.nodes[node] for node in scaffold.graph.nodes]

Of course, neurons are very flexible, and this lets us generalize easily.

In [ ]:
class even_better_AND_OR(Brick):
    def __init__(self, mode='AND',name=None):   #A change here
        super(Brick, self).__init__()
        #The brick hasn't been built yet.
        self.is_built = False
        #Leave for compatibility, D represents the depth of the circuit.  Needs to be updated.
        self.metadata = {'D':1}  
        #We just store the name passed at construction.
        self.name = name
        #For this example, we'll let any input coding work even though the answer might not make sense.
        self.supported_codings = fugu.input_coding_types
        self.mode = mode  #A change here
    def build(self,
             graph,
             metadata,
             control_nodes,
             input_lists,
             input_codings):
        #Expect two inputs
        if len(input_codings)!=2:
            raise ValueError('Only two inputs supported.')
        #Only two supported modes, AND and OR
        if self.mode is not 'AND' and self.mode is not 'OR':
            raise ValueError('Unsupported mode.')
        #Keep the same coding as input 0 for the output
        #This is an arbitrary decision at this point.  
        #Generally, your brick will impart some coding, but that isn't the case here.
        output_codings = [input_codings[0]]
        
        #All bricks should provide a neuron that spikes when the brick has completed processing.
        #We just put in a basic relay neuron that will spike when it recieves any spike from its
        #single input, which is the complete_node from the first input.
        #All nodes we add to the graph should have basic neuron parameters (threshold, decay)
        #Reasonable defaults will be filled-in, but these defaults may depend on the execution platform.
        #Additionally, nodes should have a field called 'index' which is a local index used to reference the 
        #position of the node.  This can be used by downstream bricks.  A simple example might be
        #a 3-bit binary representation will add 3 nodes to the graph with indices 0,1,2
        #We do have to do some work to establish best practices here.
        new_complete_node_name = self.name + '_complete'
        graph.add_node(new_complete_node_name,
                      index = -1,
                      threshold = 0.0,
                      decay =0.0,
                      p=1.0,
                      potential=0.0)
        graph.add_edge(control_nodes[0]['complete'], new_complete_node_name,weight=1.0,delay=1)
        

        output_lists = [[]]
        threshold_value = 1.0 if self.mode is 'AND' else 0.5
        #We also, obviously, need to build the computational portion of our graph
        for operand0 in input_lists[0]:
            for idx_num, operand1 in enumerate(input_lists[1]):
                #If indices match, we'll do an AND on them
                if graph.nodes[operand0]['index'] == graph.nodes[operand1]['index']:
                    #Remember all of our output neurons need to be marked
                    and_node_name = self.name+ '_' + str(operand0) + '_' +str(operand1)
                    output_lists[0].append(and_node_name)
                    graph.add_node(and_node_name,
                       index=0,
                       threshold=threshold_value,
                       decay=1.0,
                       p=1.0,
                       potential=0.0
                      )
                    graph.add_edge(operand0,
                                  and_node_name,
                                  weight=0.75,
                                  delay=1.0)
                    graph.add_edge(operand1,
                                  and_node_name,
                                  weight=0.75,
                                  delay=1.0)
        self.is_built=True
        
        
        return (graph,
               self.metadata,
                [{'complete':new_complete_node_name}],
                output_lists,
                output_codings
               )

In [ ]:
scaffold = Scaffold()
scaffold.add_brick(Spike_Input(np.array([1,0,1,0]), coding='Raster', name='Input0'), 'input' )
scaffold.add_brick(Spike_Input(np.array([1,1,0,0]), coding='Raster', name='Input1'), 'input' )
scaffold.add_brick(even_better_AND_OR(name='AND',mode='AND'), [(0,0), (1,0)], output=True)
scaffold.add_brick(even_better_AND_OR(name='OR', mode='OR'), [(0,0), (1,0)], output=True)
scaffold.lay_bricks()
result = scaffold.evaluate(backend='ds')
print(result)